# Data Validation Techniques

Data validation is a critical component of any data pipeline, ensuring that data meets quality standards before processing. This notebook covers comprehensive validation approaches including schema validation, business rules, statistical tests, and anomaly detection.

## Why Data Validation Matters

- **Garbage In, Garbage Out**: Poor quality data leads to unreliable analytics and ML models
- **Early Detection**: Catch issues at ingestion rather than downstream
- **Compliance**: Meet regulatory requirements (GDPR, HIPAA, SOX)
- **Trust**: Build confidence in data-driven decisions

## Validation Approaches Overview

| Approach | Purpose | When to Use |
|----------|---------|-------------|
| **Schema Validation** | Verify structure, types, constraints | Every data ingestion |
| **Business Rules** | Enforce domain-specific logic | After schema validation |
| **Statistical Tests** | Detect distribution shifts, outliers | Batch processing, monitoring |
| **Anomaly Detection** | Identify unexpected patterns | Continuous monitoring |
| **Referential Integrity** | Verify relationships between datasets | Multi-source pipelines |

In [ ]:
# Required Libraries
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from typing import List, Dict, Any, Callable, Optional, Union
from enum import Enum
from datetime import datetime, timedelta
import json
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

---

## 1. Schema Validation

Schema validation ensures data conforms to expected structure, data types, and constraints. It's the first line of defense in data quality.

### Key Components
- **Data Types**: Ensure columns have correct types (int, float, string, datetime)
- **Nullability**: Define which fields can contain null values
- **Constraints**: Value ranges, string lengths, patterns
- **Required Fields**: Mandatory columns that must be present

In [ ]:
class DataType(Enum):
    """Supported data types for validation."""
    STRING = "string"
    INTEGER = "integer"
    FLOAT = "float"
    BOOLEAN = "boolean"
    DATETIME = "datetime"
    DATE = "date"
    EMAIL = "email"
    UUID = "uuid"


@dataclass
class ColumnSchema:
    """Schema definition for a single column."""
    name: str
    dtype: DataType
    nullable: bool = True
    min_value: Optional[Union[int, float]] = None
    max_value: Optional[Union[int, float]] = None
    min_length: Optional[int] = None
    max_length: Optional[int] = None
    pattern: Optional[str] = None  # Regex pattern
    allowed_values: Optional[List[Any]] = None
    unique: bool = False


@dataclass
class TableSchema:
    """Schema definition for a complete table/dataframe."""
    name: str
    columns: List[ColumnSchema]
    primary_key: Optional[List[str]] = None
    min_rows: Optional[int] = None
    max_rows: Optional[int] = None
    
    def get_column(self, name: str) -> Optional[ColumnSchema]:
        """Get column schema by name."""
        for col in self.columns:
            if col.name == name:
                return col
        return None
    
    @property
    def required_columns(self) -> List[str]:
        """Get list of non-nullable columns."""
        return [col.name for col in self.columns if not col.nullable]

In [ ]:
@dataclass
class ValidationResult:
    """Result of a validation check."""
    is_valid: bool
    rule_name: str
    column: Optional[str] = None
    message: str = ""
    failed_count: int = 0
    total_count: int = 0
    failed_indices: List[int] = field(default_factory=list)
    severity: str = "ERROR"  # ERROR, WARNING, INFO
    
    @property
    def pass_rate(self) -> float:
        """Calculate pass rate as percentage."""
        if self.total_count == 0:
            return 100.0
        return ((self.total_count - self.failed_count) / self.total_count) * 100
    
    def __repr__(self):
        status = "✓ PASS" if self.is_valid else "✗ FAIL"
        col_info = f" [{self.column}]" if self.column else ""
        return f"{status}{col_info}: {self.rule_name} - {self.message}"


@dataclass 
class ValidationReport:
    """Aggregated validation report."""
    results: List[ValidationResult]
    validated_at: datetime = field(default_factory=datetime.now)
    
    @property
    def is_valid(self) -> bool:
        """Check if all validations passed."""
        return all(r.is_valid for r in self.results if r.severity == "ERROR")
    
    @property
    def error_count(self) -> int:
        return sum(1 for r in self.results if not r.is_valid and r.severity == "ERROR")
    
    @property
    def warning_count(self) -> int:
        return sum(1 for r in self.results if not r.is_valid and r.severity == "WARNING")
    
    def summary(self) -> str:
        """Generate summary report."""
        lines = [
            f"\n{'='*60}",
            f"VALIDATION REPORT - {self.validated_at.strftime('%Y-%m-%d %H:%M:%S')}",
            f"{'='*60}",
            f"Overall Status: {'✓ PASSED' if self.is_valid else '✗ FAILED'}",
            f"Total Checks: {len(self.results)}",
            f"Errors: {self.error_count} | Warnings: {self.warning_count}",
            f"{'-'*60}"
        ]
        
        for result in self.results:
            lines.append(str(result))
        
        lines.append(f"{'='*60}\n")
        return "\n".join(lines)

In [ ]:
import re

class SchemaValidator:
    """Validates dataframes against defined schemas."""
    
    # Type mapping for pandas dtype checking
    TYPE_MAPPING = {
        DataType.STRING: ['object', 'string', 'str'],
        DataType.INTEGER: ['int64', 'int32', 'int16', 'int8', 'Int64'],
        DataType.FLOAT: ['float64', 'float32', 'float16'],
        DataType.BOOLEAN: ['bool', 'boolean'],
        DataType.DATETIME: ['datetime64[ns]', 'datetime64'],
        DataType.DATE: ['datetime64[ns]', 'object'],
    }
    
    # Regex patterns for special types
    PATTERNS = {
        DataType.EMAIL: r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$',
        DataType.UUID: r'^[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}$'
    }
    
    def __init__(self, schema: TableSchema):
        self.schema = schema
        self.results: List[ValidationResult] = []
    
    def validate(self, df: pd.DataFrame) -> ValidationReport:
        """Run all schema validations."""
        self.results = []
        
        # Table-level validations
        self._validate_required_columns(df)
        self._validate_row_count(df)
        self._validate_primary_key(df)
        
        # Column-level validations
        for col_schema in self.schema.columns:
            if col_schema.name in df.columns:
                self._validate_column(df, col_schema)
        
        return ValidationReport(results=self.results)
    
    def _validate_required_columns(self, df: pd.DataFrame):
        """Check all required columns are present."""
        expected = {col.name for col in self.schema.columns}
        actual = set(df.columns)
        missing = expected - actual
        
        self.results.append(ValidationResult(
            is_valid=len(missing) == 0,
            rule_name="required_columns",
            message=f"Missing columns: {missing}" if missing else "All required columns present",
            failed_count=len(missing),
            total_count=len(expected)
        ))
    
    def _validate_row_count(self, df: pd.DataFrame):
        """Validate row count constraints."""
        row_count = len(df)
        is_valid = True
        messages = []
        
        if self.schema.min_rows and row_count < self.schema.min_rows:
            is_valid = False
            messages.append(f"Row count {row_count} < minimum {self.schema.min_rows}")
        
        if self.schema.max_rows and row_count > self.schema.max_rows:
            is_valid = False
            messages.append(f"Row count {row_count} > maximum {self.schema.max_rows}")
        
        self.results.append(ValidationResult(
            is_valid=is_valid,
            rule_name="row_count",
            message="; ".join(messages) if messages else f"Row count {row_count} within bounds",
            total_count=row_count
        ))
    
    def _validate_primary_key(self, df: pd.DataFrame):
        """Validate primary key uniqueness."""
        if not self.schema.primary_key:
            return
        
        pk_cols = [c for c in self.schema.primary_key if c in df.columns]
        if len(pk_cols) != len(self.schema.primary_key):
            return  # Missing PK columns handled elsewhere
        
        duplicates = df.duplicated(subset=pk_cols, keep=False)
        dup_count = duplicates.sum()
        
        self.results.append(ValidationResult(
            is_valid=dup_count == 0,
            rule_name="primary_key_unique",
            message=f"{dup_count} duplicate primary key values" if dup_count > 0 else "Primary key is unique",
            failed_count=dup_count,
            total_count=len(df),
            failed_indices=df[duplicates].index.tolist()[:100]  # Limit to first 100
        ))
    
    def _validate_column(self, df: pd.DataFrame, col_schema: ColumnSchema):
        """Validate a single column against its schema."""
        col_data = df[col_schema.name]
        
        # Null check
        self._validate_nullability(col_data, col_schema)
        
        # Type check
        self._validate_dtype(col_data, col_schema)
        
        # Range check
        if col_schema.min_value is not None or col_schema.max_value is not None:
            self._validate_range(col_data, col_schema)
        
        # Length check (for strings)
        if col_schema.dtype == DataType.STRING:
            if col_schema.min_length or col_schema.max_length:
                self._validate_length(col_data, col_schema)
        
        # Pattern check
        if col_schema.pattern or col_schema.dtype in self.PATTERNS:
            self._validate_pattern(col_data, col_schema)
        
        # Allowed values check
        if col_schema.allowed_values:
            self._validate_allowed_values(col_data, col_schema)
        
        # Uniqueness check
        if col_schema.unique:
            self._validate_uniqueness(col_data, col_schema)
    
    def _validate_nullability(self, col_data: pd.Series, col_schema: ColumnSchema):
        """Check for null values in non-nullable columns."""
        null_count = col_data.isna().sum()
        
        if not col_schema.nullable:
            self.results.append(ValidationResult(
                is_valid=null_count == 0,
                rule_name="not_null",
                column=col_schema.name,
                message=f"{null_count} null values found" if null_count > 0 else "No null values",
                failed_count=null_count,
                total_count=len(col_data)
            ))
    
    def _validate_dtype(self, col_data: pd.Series, col_schema: ColumnSchema):
        """Validate column data type."""
        actual_dtype = str(col_data.dtype)
        expected_types = self.TYPE_MAPPING.get(col_schema.dtype, [])
        
        is_valid = any(actual_dtype.startswith(t) for t in expected_types)
        
        self.results.append(ValidationResult(
            is_valid=is_valid,
            rule_name="data_type",
            column=col_schema.name,
            message=f"Expected {col_schema.dtype.value}, got {actual_dtype}" if not is_valid else f"Type {actual_dtype} valid",
            severity="WARNING" if is_valid else "ERROR"
        ))
    
    def _validate_range(self, col_data: pd.Series, col_schema: ColumnSchema):
        """Validate numeric range constraints."""
        non_null = col_data.dropna()
        failed_mask = pd.Series([False] * len(non_null), index=non_null.index)
        
        if col_schema.min_value is not None:
            failed_mask |= non_null < col_schema.min_value
        if col_schema.max_value is not None:
            failed_mask |= non_null > col_schema.max_value
        
        failed_count = failed_mask.sum()
        
        self.results.append(ValidationResult(
            is_valid=failed_count == 0,
            rule_name="value_range",
            column=col_schema.name,
            message=f"{failed_count} values outside range [{col_schema.min_value}, {col_schema.max_value}]",
            failed_count=failed_count,
            total_count=len(non_null)
        ))
    
    def _validate_length(self, col_data: pd.Series, col_schema: ColumnSchema):
        """Validate string length constraints."""
        non_null = col_data.dropna().astype(str)
        lengths = non_null.str.len()
        failed_mask = pd.Series([False] * len(lengths), index=lengths.index)
        
        if col_schema.min_length:
            failed_mask |= lengths < col_schema.min_length
        if col_schema.max_length:
            failed_mask |= lengths > col_schema.max_length
        
        failed_count = failed_mask.sum()
        
        self.results.append(ValidationResult(
            is_valid=failed_count == 0,
            rule_name="string_length",
            column=col_schema.name,
            message=f"{failed_count} values with invalid length",
            failed_count=failed_count,
            total_count=len(non_null)
        ))
    
    def _validate_pattern(self, col_data: pd.Series, col_schema: ColumnSchema):
        """Validate string pattern (regex)."""
        pattern = col_schema.pattern or self.PATTERNS.get(col_schema.dtype)
        if not pattern:
            return
        
        non_null = col_data.dropna().astype(str)
        matches = non_null.str.match(pattern)
        failed_count = (~matches).sum()
        
        self.results.append(ValidationResult(
            is_valid=failed_count == 0,
            rule_name="pattern_match",
            column=col_schema.name,
            message=f"{failed_count} values don't match pattern",
            failed_count=failed_count,
            total_count=len(non_null)
        ))
    
    def _validate_allowed_values(self, col_data: pd.Series, col_schema: ColumnSchema):
        """Validate against allowed values list."""
        non_null = col_data.dropna()
        invalid = ~non_null.isin(col_schema.allowed_values)
        failed_count = invalid.sum()
        
        self.results.append(ValidationResult(
            is_valid=failed_count == 0,
            rule_name="allowed_values",
            column=col_schema.name,
            message=f"{failed_count} values not in allowed list",
            failed_count=failed_count,
            total_count=len(non_null)
        ))
    
    def _validate_uniqueness(self, col_data: pd.Series, col_schema: ColumnSchema):
        """Validate column uniqueness."""
        duplicates = col_data.duplicated(keep=False)
        dup_count = duplicates.sum()
        
        self.results.append(ValidationResult(
            is_valid=dup_count == 0,
            rule_name="unique",
            column=col_schema.name,
            message=f"{dup_count} duplicate values" if dup_count > 0 else "All values unique",
            failed_count=dup_count,
            total_count=len(col_data)
        ))

In [ ]:
# Example: Schema Validation in Action

# Define schema for customer data
customer_schema = TableSchema(
    name="customers",
    columns=[
        ColumnSchema("customer_id", DataType.STRING, nullable=False, unique=True),
        ColumnSchema("email", DataType.EMAIL, nullable=False),
        ColumnSchema("age", DataType.INTEGER, nullable=True, min_value=0, max_value=150),
        ColumnSchema("status", DataType.STRING, allowed_values=["active", "inactive", "pending"]),
        ColumnSchema("credit_score", DataType.INTEGER, min_value=300, max_value=850),
    ],
    primary_key=["customer_id"],
    min_rows=1
)

# Sample data with some issues
sample_data = pd.DataFrame({
    "customer_id": ["C001", "C002", "C003", "C002", "C005"],  # Duplicate C002
    "email": ["john@example.com", "invalid-email", "jane@test.org", "bob@company.com", "alice@corp.net"],
    "age": [25, 35, -5, 45, 200],  # Invalid: -5 and 200
    "status": ["active", "inactive", "pending", "unknown", "active"],  # Invalid: unknown
    "credit_score": [720, 650, 800, 250, 780]  # Invalid: 250 below min
})

# Run validation
validator = SchemaValidator(customer_schema)
report = validator.validate(sample_data)
print(report.summary())

---

## 2. Business Rules Validation

Business rules go beyond schema validation to enforce domain-specific logic and complex conditions that span multiple columns or require external context.

### Common Business Rules
- **Cross-field validation**: Start date must be before end date
- **Conditional requirements**: If status is 'completed', completion_date is required
- **Computed validations**: Total must equal sum of components
- **External lookups**: Product ID must exist in master catalog

In [ ]:
class BusinessRule(ABC):
    """Abstract base class for business rules."""
    
    def __init__(self, name: str, severity: str = "ERROR"):
        self.name = name
        self.severity = severity
    
    @abstractmethod
    def validate(self, df: pd.DataFrame) -> ValidationResult:
        """Execute the validation rule."""
        pass


class CrossFieldRule(BusinessRule):
    """Validates relationships between multiple columns."""
    
    def __init__(self, name: str, condition: Callable[[pd.DataFrame], pd.Series], 
                 message: str, severity: str = "ERROR"):
        super().__init__(name, severity)
        self.condition = condition
        self.message = message
    
    def validate(self, df: pd.DataFrame) -> ValidationResult:
        try:
            # Condition should return True for valid rows
            valid_mask = self.condition(df)
            failed_count = (~valid_mask).sum()
            
            return ValidationResult(
                is_valid=failed_count == 0,
                rule_name=self.name,
                message=f"{failed_count} rows failed: {self.message}" if failed_count > 0 else "Rule passed",
                failed_count=failed_count,
                total_count=len(df),
                failed_indices=df[~valid_mask].index.tolist()[:100],
                severity=self.severity
            )
        except Exception as e:
            return ValidationResult(
                is_valid=False,
                rule_name=self.name,
                message=f"Rule execution error: {str(e)}",
                severity="ERROR"
            )


class ConditionalRequiredRule(BusinessRule):
    """Field is required when condition is met."""
    
    def __init__(self, name: str, condition_field: str, condition_value: Any,
                 required_field: str, severity: str = "ERROR"):
        super().__init__(name, severity)
        self.condition_field = condition_field
        self.condition_value = condition_value
        self.required_field = required_field
    
    def validate(self, df: pd.DataFrame) -> ValidationResult:
        # Find rows where condition is met but required field is null
        condition_met = df[self.condition_field] == self.condition_value
        required_null = df[self.required_field].isna()
        failed = condition_met & required_null
        failed_count = failed.sum()
        
        return ValidationResult(
            is_valid=failed_count == 0,
            rule_name=self.name,
            message=f"{self.required_field} required when {self.condition_field}={self.condition_value}",
            failed_count=failed_count,
            total_count=condition_met.sum(),
            failed_indices=df[failed].index.tolist(),
            severity=self.severity
        )


class SumValidationRule(BusinessRule):
    """Validates that a total column equals sum of component columns."""
    
    def __init__(self, name: str, total_column: str, component_columns: List[str],
                 tolerance: float = 0.01, severity: str = "ERROR"):
        super().__init__(name, severity)
        self.total_column = total_column
        self.component_columns = component_columns
        self.tolerance = tolerance
    
    def validate(self, df: pd.DataFrame) -> ValidationResult:
        calculated_sum = df[self.component_columns].sum(axis=1)
        actual_total = df[self.total_column]
        
        # Check if difference is within tolerance
        diff = (calculated_sum - actual_total).abs()
        failed = diff > self.tolerance
        failed_count = failed.sum()
        
        return ValidationResult(
            is_valid=failed_count == 0,
            rule_name=self.name,
            column=self.total_column,
            message=f"{failed_count} rows where total doesn't match sum of components",
            failed_count=failed_count,
            total_count=len(df),
            failed_indices=df[failed].index.tolist(),
            severity=self.severity
        )


class ReferentialIntegrityRule(BusinessRule):
    """Validates foreign key references exist in parent table."""
    
    def __init__(self, name: str, fk_column: str, reference_values: set,
                 severity: str = "ERROR"):
        super().__init__(name, severity)
        self.fk_column = fk_column
        self.reference_values = reference_values
    
    def validate(self, df: pd.DataFrame) -> ValidationResult:
        non_null = df[self.fk_column].dropna()
        invalid = ~non_null.isin(self.reference_values)
        failed_count = invalid.sum()
        
        return ValidationResult(
            is_valid=failed_count == 0,
            rule_name=self.name,
            column=self.fk_column,
            message=f"{failed_count} values don't exist in reference table",
            failed_count=failed_count,
            total_count=len(non_null),
            severity=self.severity
        )

In [ ]:
class BusinessRuleValidator:
    """Orchestrates multiple business rule validations."""
    
    def __init__(self):
        self.rules: List[BusinessRule] = []
    
    def add_rule(self, rule: BusinessRule) -> 'BusinessRuleValidator':
        """Add a rule (fluent interface)."""
        self.rules.append(rule)
        return self
    
    def validate(self, df: pd.DataFrame) -> ValidationReport:
        """Execute all business rules."""
        results = [rule.validate(df) for rule in self.rules]
        return ValidationReport(results=results)


# Example: Business Rules Validation
order_data = pd.DataFrame({
    "order_id": ["O001", "O002", "O003", "O004", "O005"],
    "order_date": pd.to_datetime(["2024-01-01", "2024-02-01", "2024-03-01", "2024-04-01", "2024-05-01"]),
    "ship_date": pd.to_datetime(["2024-01-05", "2024-01-15", "2024-02-28", None, "2024-05-10"]),
    "status": ["shipped", "shipped", "pending", "shipped", "delivered"],
    "subtotal": [100.0, 200.0, 150.0, 300.0, 250.0],
    "tax": [8.0, 16.0, 12.0, 24.0, 20.0],
    "shipping": [10.0, 15.0, 12.0, 20.0, 18.0],
    "total": [118.0, 230.0, 174.0, 344.0, 288.0],  # O002 is wrong (should be 231)
    "product_id": ["P001", "P002", "P003", "P999", "P001"]  # P999 doesn't exist
})

# Valid product IDs (simulating lookup table)
valid_products = {"P001", "P002", "P003", "P004", "P005"}

# Define business rules
br_validator = BusinessRuleValidator()
br_validator.add_rule(
    CrossFieldRule(
        name="ship_after_order",
        condition=lambda df: df["ship_date"].isna() | (df["ship_date"] >= df["order_date"]),
        message="Ship date must be on or after order date"
    )
).add_rule(
    ConditionalRequiredRule(
        name="shipped_requires_ship_date",
        condition_field="status",
        condition_value="shipped",
        required_field="ship_date"
    )
).add_rule(
    SumValidationRule(
        name="total_calculation",
        total_column="total",
        component_columns=["subtotal", "tax", "shipping"],
        tolerance=0.01
    )
).add_rule(
    ReferentialIntegrityRule(
        name="valid_product",
        fk_column="product_id",
        reference_values=valid_products
    )
)

# Run validation
br_report = br_validator.validate(order_data)
print(br_report.summary())

---

## 3. Statistical Validation Tests

Statistical tests detect distribution shifts, outliers, and data drift that schema/business rules can't catch. These are essential for maintaining data quality over time.

### Key Statistical Tests
- **Distribution Tests**: Kolmogorov-Smirnov, Chi-Square for categorical
- **Outlier Detection**: Z-score, IQR, Modified Z-score
- **Data Drift**: Population Stability Index (PSI)
- **Completeness**: Null rates, coverage metrics

In [ ]:
from scipy import stats

class StatisticalValidator:
    """Statistical validation tests for data quality."""
    
    def __init__(self, reference_df: Optional[pd.DataFrame] = None):
        """
        Initialize with optional reference data for comparison.
        
        Args:
            reference_df: Baseline data for distribution comparison
        """
        self.reference_df = reference_df
        self.results: List[ValidationResult] = []
    
    def validate(self, df: pd.DataFrame, columns: Optional[List[str]] = None) -> ValidationReport:
        """Run all statistical validations."""
        self.results = []
        
        cols = columns or df.select_dtypes(include=[np.number]).columns.tolist()
        
        for col in cols:
            if col in df.columns:
                self._validate_column_stats(df, col)
        
        return ValidationReport(results=self.results)
    
    def _validate_column_stats(self, df: pd.DataFrame, column: str):
        """Run statistical tests on a single column."""
        col_data = df[column].dropna()
        
        if len(col_data) < 2:
            return
        
        # Outlier detection
        self._detect_outliers_zscore(col_data, column)
        self._detect_outliers_iqr(col_data, column)
        
        # Distribution comparison (if reference exists)
        if self.reference_df is not None and column in self.reference_df.columns:
            self._compare_distributions(col_data, column)
    
    def _detect_outliers_zscore(self, data: pd.Series, column: str, 
                                 threshold: float = 3.0):
        """Detect outliers using Z-score method."""
        mean = data.mean()
        std = data.std()
        
        if std == 0:
            return
        
        z_scores = np.abs((data - mean) / std)
        outliers = z_scores > threshold
        outlier_count = outliers.sum()
        outlier_pct = (outlier_count / len(data)) * 100
        
        # Warning if outliers exceed 5% of data
        self.results.append(ValidationResult(
            is_valid=outlier_pct <= 5.0,
            rule_name="zscore_outliers",
            column=column,
            message=f"{outlier_count} outliers ({outlier_pct:.1f}%) detected (|Z| > {threshold})",
            failed_count=outlier_count,
            total_count=len(data),
            severity="WARNING"
        ))
    
    def _detect_outliers_iqr(self, data: pd.Series, column: str,
                             multiplier: float = 1.5):
        """Detect outliers using IQR method."""
        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        iqr = q3 - q1
        
        lower_bound = q1 - multiplier * iqr
        upper_bound = q3 + multiplier * iqr
        
        outliers = (data < lower_bound) | (data > upper_bound)
        outlier_count = outliers.sum()
        outlier_pct = (outlier_count / len(data)) * 100
        
        self.results.append(ValidationResult(
            is_valid=outlier_pct <= 5.0,
            rule_name="iqr_outliers",
            column=column,
            message=f"{outlier_count} outliers ({outlier_pct:.1f}%) outside [{lower_bound:.2f}, {upper_bound:.2f}]",
            failed_count=outlier_count,
            total_count=len(data),
            severity="WARNING"
        ))
    
    def _compare_distributions(self, data: pd.Series, column: str,
                               alpha: float = 0.05):
        """Compare distribution with reference using KS test."""
        ref_data = self.reference_df[column].dropna()
        
        if len(ref_data) < 2:
            return
        
        # Kolmogorov-Smirnov test
        ks_stat, p_value = stats.ks_2samp(data, ref_data)
        
        self.results.append(ValidationResult(
            is_valid=p_value >= alpha,
            rule_name="distribution_drift",
            column=column,
            message=f"KS statistic: {ks_stat:.4f}, p-value: {p_value:.4f}" + 
                    (" - Distribution drift detected!" if p_value < alpha else " - No significant drift"),
            severity="WARNING" if p_value >= alpha else "ERROR"
        ))
    
    @staticmethod
    def calculate_psi(expected: pd.Series, actual: pd.Series, 
                      buckets: int = 10) -> float:
        """
        Calculate Population Stability Index (PSI).
        
        PSI < 0.1: No significant change
        0.1 <= PSI < 0.2: Moderate change
        PSI >= 0.2: Significant change
        """
        # Create buckets based on expected distribution
        breakpoints = np.percentile(expected.dropna(), 
                                    np.linspace(0, 100, buckets + 1))
        breakpoints = np.unique(breakpoints)
        
        # Calculate proportions in each bucket
        expected_counts = np.histogram(expected.dropna(), bins=breakpoints)[0]
        actual_counts = np.histogram(actual.dropna(), bins=breakpoints)[0]
        
        # Normalize to proportions
        expected_pct = expected_counts / expected_counts.sum()
        actual_pct = actual_counts / actual_counts.sum()
        
        # Avoid division by zero
        expected_pct = np.where(expected_pct == 0, 0.0001, expected_pct)
        actual_pct = np.where(actual_pct == 0, 0.0001, actual_pct)
        
        # Calculate PSI
        psi = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
        
        return psi

In [ ]:
# Example: Statistical Validation
np.random.seed(42)

# Reference/baseline data (training period)
reference_data = pd.DataFrame({
    "transaction_amount": np.random.lognormal(4, 1, 1000),
    "customer_age": np.random.normal(35, 10, 1000),
    "purchase_frequency": np.random.poisson(5, 1000)
})

# Current data with some drift
current_data = pd.DataFrame({
    "transaction_amount": np.concatenate([
        np.random.lognormal(4, 1, 900),
        np.random.lognormal(6, 1, 100)  # 10% shifted distribution
    ]),
    "customer_age": np.random.normal(40, 12, 1000),  # Mean shift
    "purchase_frequency": np.random.poisson(5, 1000)  # No change
})

# Add some outliers
current_data.loc[0, "transaction_amount"] = 50000  # Extreme outlier
current_data.loc[1, "transaction_amount"] = 45000

# Run statistical validation
stat_validator = StatisticalValidator(reference_df=reference_data)
stat_report = stat_validator.validate(current_data)
print(stat_report.summary())

# Calculate PSI for transaction amounts
psi = StatisticalValidator.calculate_psi(
    reference_data["transaction_amount"],
    current_data["transaction_amount"]
)
print(f"\nPSI for transaction_amount: {psi:.4f}")
print(f"Interpretation: {'No significant change' if psi < 0.1 else 'Moderate change' if psi < 0.2 else 'Significant change!'}")

---

## 4. Anomaly Detection for Data Quality

Anomaly detection identifies unusual patterns that may indicate data quality issues, fraud, or system problems. Unlike rule-based validation, anomaly detection learns patterns from data.

### Anomaly Detection Approaches
- **Univariate**: Extreme values in single columns
- **Multivariate**: Unusual combinations of values
- **Time-series**: Temporal anomalies, seasonality violations
- **Categorical**: Rare category occurrences

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor


class AnomalyDetector:
    """Multi-method anomaly detection for data quality."""
    
    def __init__(self, contamination: float = 0.05):
        """
        Initialize anomaly detector.
        
        Args:
            contamination: Expected proportion of anomalies (0-0.5)
        """
        self.contamination = contamination
        self.scaler = StandardScaler()
        self.isolation_forest = None
        self.lof = None
    
    def detect_isolation_forest(self, df: pd.DataFrame, 
                                 columns: List[str]) -> pd.DataFrame:
        """
        Detect anomalies using Isolation Forest.
        
        Isolation Forest works by randomly selecting features and split values,
        isolating observations. Anomalies are isolated quickly (short path length).
        """
        data = df[columns].dropna()
        scaled_data = self.scaler.fit_transform(data)
        
        self.isolation_forest = IsolationForest(
            contamination=self.contamination,
            random_state=42,
            n_estimators=100
        )
        
        # Predict: -1 for anomalies, 1 for normal
        predictions = self.isolation_forest.fit_predict(scaled_data)
        scores = self.isolation_forest.decision_function(scaled_data)
        
        result = df.loc[data.index].copy()
        result['is_anomaly_if'] = predictions == -1
        result['anomaly_score_if'] = -scores  # Higher = more anomalous
        
        return result
    
    def detect_lof(self, df: pd.DataFrame, columns: List[str],
                   n_neighbors: int = 20) -> pd.DataFrame:
        """
        Detect anomalies using Local Outlier Factor.
        
        LOF measures local density deviation. Points with substantially lower
        density than neighbors are considered outliers.
        """
        data = df[columns].dropna()
        scaled_data = self.scaler.fit_transform(data)
        
        self.lof = LocalOutlierFactor(
            n_neighbors=n_neighbors,
            contamination=self.contamination
        )
        
        predictions = self.lof.fit_predict(scaled_data)
        scores = -self.lof.negative_outlier_factor_
        
        result = df.loc[data.index].copy()
        result['is_anomaly_lof'] = predictions == -1
        result['anomaly_score_lof'] = scores
        
        return result
    
    def detect_mahalanobis(self, df: pd.DataFrame, 
                           columns: List[str]) -> pd.DataFrame:
        """
        Detect anomalies using Mahalanobis distance.
        
        Mahalanobis distance accounts for correlations between variables
        and variable scales.
        """
        data = df[columns].dropna()
        
        # Calculate mean and covariance
        mean = data.mean()
        cov = data.cov()
        
        try:
            cov_inv = np.linalg.inv(cov)
        except np.linalg.LinAlgError:
            # Use pseudo-inverse if singular
            cov_inv = np.linalg.pinv(cov)
        
        # Calculate Mahalanobis distance for each row
        diff = data - mean
        mahal_dist = np.sqrt(
            np.sum(np.dot(diff, cov_inv) * diff, axis=1)
        )
        
        # Chi-square threshold (p < 0.001)
        threshold = np.sqrt(stats.chi2.ppf(0.999, df=len(columns)))
        
        result = df.loc[data.index].copy()
        result['mahalanobis_dist'] = mahal_dist
        result['is_anomaly_mahal'] = mahal_dist > threshold
        
        return result
    
    def ensemble_detection(self, df: pd.DataFrame, 
                           columns: List[str],
                           min_votes: int = 2) -> pd.DataFrame:
        """
        Ensemble anomaly detection combining multiple methods.
        
        Args:
            df: Input dataframe
            columns: Columns to use for detection
            min_votes: Minimum methods that must flag as anomaly
        """
        result = df.copy()
        
        # Run all methods
        if_result = self.detect_isolation_forest(df, columns)
        lof_result = self.detect_lof(df, columns)
        mahal_result = self.detect_mahalanobis(df, columns)
        
        # Combine results
        valid_idx = if_result.index
        result = result.loc[valid_idx].copy()
        
        result['votes_anomaly'] = (
            if_result['is_anomaly_if'].astype(int) +
            lof_result['is_anomaly_lof'].astype(int) +
            mahal_result['is_anomaly_mahal'].astype(int)
        )
        
        result['is_anomaly_ensemble'] = result['votes_anomaly'] >= min_votes
        result['anomaly_score_if'] = if_result['anomaly_score_if']
        result['anomaly_score_lof'] = lof_result['anomaly_score_lof']
        result['mahalanobis_dist'] = mahal_result['mahalanobis_dist']
        
        return result

In [ ]:
# Example: Anomaly Detection
np.random.seed(42)

# Generate normal transaction data
n_normal = 950
normal_data = pd.DataFrame({
    "amount": np.random.lognormal(4, 0.5, n_normal),
    "frequency": np.random.poisson(3, n_normal),
    "avg_basket_size": np.random.normal(5, 1.5, n_normal)
})

# Add anomalies
n_anomalies = 50
anomaly_data = pd.DataFrame({
    "amount": np.random.lognormal(6, 0.8, n_anomalies),  # Higher amounts
    "frequency": np.random.poisson(15, n_anomalies),     # Higher frequency
    "avg_basket_size": np.random.normal(2, 0.5, n_anomalies)  # Lower basket
})

# Combine and shuffle
full_data = pd.concat([normal_data, anomaly_data], ignore_index=True)
full_data = full_data.sample(frac=1, random_state=42).reset_index(drop=True)

# True labels (for evaluation)
true_labels = np.concatenate([np.zeros(n_normal), np.ones(n_anomalies)])

# Run ensemble detection
detector = AnomalyDetector(contamination=0.05)
results = detector.ensemble_detection(
    full_data, 
    columns=["amount", "frequency", "avg_basket_size"]
)

# Display top anomalies
anomalies = results[results['is_anomaly_ensemble']].sort_values(
    'votes_anomaly', ascending=False
)
print(f"Detected {len(anomalies)} anomalies using ensemble method\n")
print("Top 10 anomalies:")
display(anomalies.head(10))

---

## 5. Pipeline Integration

Integrating validation into data pipelines ensures quality checks happen automatically at critical points. Here we build a complete validation pipeline framework.

### Integration Patterns
- **Gate Validation**: Block pipeline if validation fails
- **Alert Validation**: Continue but notify on issues
- **Quarantine Pattern**: Route bad records to separate storage
- **Enrichment Pattern**: Add validation metadata to records

In [ ]:
from enum import Enum
from typing import Tuple


class ValidationAction(Enum):
    """Actions to take on validation failure."""
    GATE = "gate"          # Stop pipeline
    ALERT = "alert"        # Continue with alerts
    QUARANTINE = "quarantine"  # Separate bad records
    ENRICH = "enrich"      # Add metadata and continue


@dataclass
class ValidationStage:
    """A stage in the validation pipeline."""
    name: str
    validator: Any  # SchemaValidator, BusinessRuleValidator, etc.
    action: ValidationAction = ValidationAction.GATE
    threshold: float = 1.0  # Pass rate threshold (0-1)
    

class DataValidationPipeline:
    """
    Comprehensive data validation pipeline.
    
    Combines schema, business rule, and statistical validation
    with configurable actions on failure.
    """
    
    def __init__(self, name: str):
        self.name = name
        self.stages: List[ValidationStage] = []
        self.alerts: List[str] = []
        self.quarantine_data: Optional[pd.DataFrame] = None
    
    def add_stage(self, stage: ValidationStage) -> 'DataValidationPipeline':
        """Add a validation stage."""
        self.stages.append(stage)
        return self
    
    def run(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, ValidationReport]]:
        """
        Execute the validation pipeline.
        
        Returns:
            Tuple of (processed dataframe, dict of reports by stage)
        """
        self.alerts = []
        self.quarantine_data = pd.DataFrame()
        reports = {}
        current_df = df.copy()
        
        for stage in self.stages:
            print(f"\n▶ Running stage: {stage.name}")
            
            # Run validation
            report = stage.validator.validate(current_df)
            reports[stage.name] = report
            
            # Calculate pass rate
            pass_rate = self._calculate_pass_rate(report)
            
            # Handle based on action type
            if pass_rate < stage.threshold:
                current_df = self._handle_failure(
                    stage, report, current_df, pass_rate
                )
                
                if current_df is None:
                    # Pipeline halted
                    break
            else:
                print(f"  ✓ Passed ({pass_rate:.1%} >= {stage.threshold:.1%})")
        
        return current_df, reports
    
    def _calculate_pass_rate(self, report: ValidationReport) -> float:
        """Calculate overall pass rate from report."""
        if not report.results:
            return 1.0
        
        passed = sum(1 for r in report.results if r.is_valid)
        return passed / len(report.results)
    
    def _handle_failure(self, stage: ValidationStage, report: ValidationReport,
                        df: pd.DataFrame, pass_rate: float) -> Optional[pd.DataFrame]:
        """Handle validation failure based on action type."""
        
        if stage.action == ValidationAction.GATE:
            print(f"  ✗ GATE - Pipeline stopped ({pass_rate:.1%} < {stage.threshold:.1%})")
            self.alerts.append(f"Pipeline halted at {stage.name}")
            return None
        
        elif stage.action == ValidationAction.ALERT:
            alert = f"Validation issues in {stage.name}: {report.error_count} errors, {report.warning_count} warnings"
            self.alerts.append(alert)
            print(f"  ⚠ ALERT - {alert}")
            return df
        
        elif stage.action == ValidationAction.QUARANTINE:
            # Collect failed row indices
            failed_indices = set()
            for result in report.results:
                if not result.is_valid:
                    failed_indices.update(result.failed_indices)
            
            if failed_indices:
                quarantine = df.loc[list(failed_indices)].copy()
                quarantine['_quarantine_reason'] = stage.name
                quarantine['_quarantine_time'] = datetime.now()
                self.quarantine_data = pd.concat([self.quarantine_data, quarantine])
                
                # Remove quarantined rows
                df = df.drop(index=list(failed_indices), errors='ignore')
                print(f"  ⚡ QUARANTINE - {len(failed_indices)} rows moved to quarantine")
            
            return df
        
        elif stage.action == ValidationAction.ENRICH:
            # Add validation metadata
            df['_validation_stage'] = stage.name
            df['_validation_passed'] = pass_rate >= stage.threshold
            df['_validation_pass_rate'] = pass_rate
            df['_validation_time'] = datetime.now()
            print(f"  📝 ENRICH - Added validation metadata")
            return df
        
        return df
    
    def get_quarantined(self) -> pd.DataFrame:
        """Get all quarantined records."""
        return self.quarantine_data
    
    def get_alerts(self) -> List[str]:
        """Get all alerts generated."""
        return self.alerts

In [ ]:
# Example: Complete Validation Pipeline

# Define schemas
transaction_schema = TableSchema(
    name="transactions",
    columns=[
        ColumnSchema("transaction_id", DataType.STRING, nullable=False, unique=True),
        ColumnSchema("amount", DataType.FLOAT, nullable=False, min_value=0),
        ColumnSchema("customer_id", DataType.STRING, nullable=False),
        ColumnSchema("category", DataType.STRING, allowed_values=["retail", "food", "travel", "other"]),
    ],
    primary_key=["transaction_id"]
)

# Create sample data with issues
transactions = pd.DataFrame({
    "transaction_id": ["T001", "T002", "T003", "T004", "T005", "T006", "T002"],  # Duplicate
    "amount": [100.0, 250.0, -50.0, 1000.0, 75.0, 50000.0, 150.0],  # Negative and outlier
    "customer_id": ["C001", "C002", None, "C004", "C005", "C006", "C002"],  # Null
    "category": ["retail", "food", "travel", "invalid", "other", "retail", "food"]  # Invalid
})

# Business rules
br_validator = BusinessRuleValidator()
br_validator.add_rule(
    CrossFieldRule(
        name="high_value_check",
        condition=lambda df: df["amount"] < 10000,
        message="Amounts over $10,000 require review",
        severity="WARNING"
    )
)

# Build pipeline
pipeline = DataValidationPipeline("transaction_validation")
pipeline.add_stage(ValidationStage(
    name="schema_validation",
    validator=SchemaValidator(transaction_schema),
    action=ValidationAction.QUARANTINE,
    threshold=0.8
)).add_stage(ValidationStage(
    name="business_rules",
    validator=br_validator,
    action=ValidationAction.ALERT,
    threshold=0.9
))

# Run pipeline
result_df, reports = pipeline.run(transactions)

print("\n" + "="*60)
print("PIPELINE COMPLETE")
print("="*60)
print(f"\nInput rows: {len(transactions)}")
print(f"Output rows: {len(result_df) if result_df is not None else 0}")
print(f"Quarantined rows: {len(pipeline.get_quarantined())}")
print(f"Alerts: {len(pipeline.get_alerts())}")

if len(pipeline.get_quarantined()) > 0:
    print("\nQuarantined records:")
    display(pipeline.get_quarantined())

---

## 6. Great Expectations Integration

For production systems, **Great Expectations** is the industry-standard library for data validation. Here's how to integrate it with custom validation logic.

In [ ]:
# Great Expectations-style expectation implementation
# This mimics the GE API for those familiar with the library

class ExpectationSuite:
    """
    Lightweight Great Expectations-style validation suite.
    
    In production, use the actual great_expectations library:
    pip install great_expectations
    """
    
    def __init__(self, name: str):
        self.name = name
        self.expectations: List[Dict] = []
    
    def expect_column_to_exist(self, column: str):
        self.expectations.append({
            "expectation_type": "expect_column_to_exist",
            "kwargs": {"column": column}
        })
        return self
    
    def expect_column_values_to_not_be_null(self, column: str):
        self.expectations.append({
            "expectation_type": "expect_column_values_to_not_be_null",
            "kwargs": {"column": column}
        })
        return self
    
    def expect_column_values_to_be_between(self, column: str, 
                                           min_value: float = None, 
                                           max_value: float = None):
        self.expectations.append({
            "expectation_type": "expect_column_values_to_be_between",
            "kwargs": {"column": column, "min_value": min_value, "max_value": max_value}
        })
        return self
    
    def expect_column_values_to_be_in_set(self, column: str, value_set: List):
        self.expectations.append({
            "expectation_type": "expect_column_values_to_be_in_set",
            "kwargs": {"column": column, "value_set": value_set}
        })
        return self
    
    def expect_column_values_to_be_unique(self, column: str):
        self.expectations.append({
            "expectation_type": "expect_column_values_to_be_unique",
            "kwargs": {"column": column}
        })
        return self
    
    def validate(self, df: pd.DataFrame) -> Dict:
        """Run all expectations against dataframe."""
        results = []
        
        for exp in self.expectations:
            result = self._evaluate_expectation(df, exp)
            results.append(result)
        
        return {
            "success": all(r["success"] for r in results),
            "results": results,
            "statistics": {
                "evaluated_expectations": len(results),
                "successful_expectations": sum(1 for r in results if r["success"]),
                "unsuccessful_expectations": sum(1 for r in results if not r["success"])
            }
        }
    
    def _evaluate_expectation(self, df: pd.DataFrame, exp: Dict) -> Dict:
        """Evaluate a single expectation."""
        exp_type = exp["expectation_type"]
        kwargs = exp["kwargs"]
        
        evaluators = {
            "expect_column_to_exist": self._eval_column_exists,
            "expect_column_values_to_not_be_null": self._eval_not_null,
            "expect_column_values_to_be_between": self._eval_between,
            "expect_column_values_to_be_in_set": self._eval_in_set,
            "expect_column_values_to_be_unique": self._eval_unique,
        }
        
        evaluator = evaluators.get(exp_type)
        if evaluator:
            return evaluator(df, kwargs)
        
        return {"success": False, "message": f"Unknown expectation: {exp_type}"}
    
    def _eval_column_exists(self, df: pd.DataFrame, kwargs: Dict) -> Dict:
        col = kwargs["column"]
        exists = col in df.columns
        return {
            "expectation_type": "expect_column_to_exist",
            "success": exists,
            "result": {"observed_value": list(df.columns)}
        }
    
    def _eval_not_null(self, df: pd.DataFrame, kwargs: Dict) -> Dict:
        col = kwargs["column"]
        null_count = df[col].isna().sum()
        return {
            "expectation_type": "expect_column_values_to_not_be_null",
            "success": null_count == 0,
            "result": {"unexpected_count": int(null_count)}
        }
    
    def _eval_between(self, df: pd.DataFrame, kwargs: Dict) -> Dict:
        col = kwargs["column"]
        min_val = kwargs.get("min_value")
        max_val = kwargs.get("max_value")
        
        data = df[col].dropna()
        failed = pd.Series([False] * len(data), index=data.index)
        
        if min_val is not None:
            failed |= data < min_val
        if max_val is not None:
            failed |= data > max_val
        
        return {
            "expectation_type": "expect_column_values_to_be_between",
            "success": failed.sum() == 0,
            "result": {"unexpected_count": int(failed.sum())}
        }
    
    def _eval_in_set(self, df: pd.DataFrame, kwargs: Dict) -> Dict:
        col = kwargs["column"]
        value_set = set(kwargs["value_set"])
        
        data = df[col].dropna()
        unexpected = ~data.isin(value_set)
        
        return {
            "expectation_type": "expect_column_values_to_be_in_set",
            "success": unexpected.sum() == 0,
            "result": {"unexpected_count": int(unexpected.sum())}
        }
    
    def _eval_unique(self, df: pd.DataFrame, kwargs: Dict) -> Dict:
        col = kwargs["column"]
        duplicates = df[col].duplicated().sum()
        
        return {
            "expectation_type": "expect_column_values_to_be_unique",
            "success": duplicates == 0,
            "result": {"unexpected_count": int(duplicates)}
        }


# Example usage
suite = ExpectationSuite("customer_data_suite")
suite.expect_column_to_exist("customer_id") \
     .expect_column_values_to_not_be_null("customer_id") \
     .expect_column_values_to_be_unique("customer_id") \
     .expect_column_values_to_be_between("age", min_value=0, max_value=120) \
     .expect_column_values_to_be_in_set("status", ["active", "inactive"])

test_data = pd.DataFrame({
    "customer_id": ["C001", "C002", "C001", "C004"],  # Duplicate
    "age": [25, 150, 35, -5],  # Out of range
    "status": ["active", "inactive", "pending", "active"]  # Invalid value
})

validation_result = suite.validate(test_data)
print(json.dumps(validation_result, indent=2))

---

## 7. Monitoring and Alerting

Continuous monitoring tracks validation metrics over time, detecting degradation patterns and triggering alerts.

In [ ]:
@dataclass
class ValidationMetric:
    """A validation metric data point."""
    timestamp: datetime
    metric_name: str
    value: float
    threshold: float
    is_breach: bool
    context: Dict = field(default_factory=dict)


class ValidationMonitor:
    """
    Monitors validation metrics over time.
    
    Tracks:
    - Pass rates by validation type
    - Null rates per column
    - Anomaly detection rates
    - Schema conformance
    """
    
    def __init__(self):
        self.metrics_history: List[ValidationMetric] = []
        self.thresholds: Dict[str, float] = {
            "schema_pass_rate": 0.95,
            "business_rule_pass_rate": 0.90,
            "null_rate": 0.05,
            "anomaly_rate": 0.10,
            "duplicate_rate": 0.01
        }
    
    def record_metric(self, metric_name: str, value: float, 
                      context: Dict = None) -> ValidationMetric:
        """Record a validation metric."""
        threshold = self.thresholds.get(metric_name, 1.0)
        
        # For pass rates, breach is when below threshold
        # For rates (null, anomaly, duplicate), breach is when above threshold
        if "pass_rate" in metric_name:
            is_breach = value < threshold
        else:
            is_breach = value > threshold
        
        metric = ValidationMetric(
            timestamp=datetime.now(),
            metric_name=metric_name,
            value=value,
            threshold=threshold,
            is_breach=is_breach,
            context=context or {}
        )
        
        self.metrics_history.append(metric)
        return metric
    
    def record_from_report(self, report: ValidationReport, 
                           source: str = "unknown") -> List[ValidationMetric]:
        """Extract and record metrics from a validation report."""
        metrics = []
        
        # Overall pass rate
        total = len(report.results)
        passed = sum(1 for r in report.results if r.is_valid)
        pass_rate = passed / total if total > 0 else 1.0
        
        metrics.append(self.record_metric(
            "schema_pass_rate", pass_rate, {"source": source}
        ))
        
        # Per-column null rates
        for result in report.results:
            if result.rule_name == "not_null" and result.column:
                null_rate = result.failed_count / result.total_count if result.total_count > 0 else 0
                metrics.append(self.record_metric(
                    "null_rate", null_rate, 
                    {"source": source, "column": result.column}
                ))
        
        return metrics
    
    def get_breaches(self, since: datetime = None) -> List[ValidationMetric]:
        """Get all threshold breaches."""
        breaches = [m for m in self.metrics_history if m.is_breach]
        
        if since:
            breaches = [m for m in breaches if m.timestamp >= since]
        
        return breaches
    
    def get_trend(self, metric_name: str, periods: int = 10) -> pd.DataFrame:
        """Get trend data for a metric."""
        relevant = [m for m in self.metrics_history if m.metric_name == metric_name]
        relevant = sorted(relevant, key=lambda x: x.timestamp)[-periods:]
        
        return pd.DataFrame([
            {
                "timestamp": m.timestamp,
                "value": m.value,
                "threshold": m.threshold,
                "is_breach": m.is_breach
            }
            for m in relevant
        ])
    
    def summary(self) -> str:
        """Generate monitoring summary."""
        lines = [
            "\n" + "="*60,
            "VALIDATION MONITORING SUMMARY",
            "="*60
        ]
        
        # Group by metric name
        by_metric = {}
        for m in self.metrics_history:
            if m.metric_name not in by_metric:
                by_metric[m.metric_name] = []
            by_metric[m.metric_name].append(m)
        
        for metric_name, metrics in by_metric.items():
            values = [m.value for m in metrics]
            breaches = sum(1 for m in metrics if m.is_breach)
            
            lines.append(f"\n{metric_name}:")
            lines.append(f"  Latest: {values[-1]:.3f}")
            lines.append(f"  Average: {np.mean(values):.3f}")
            lines.append(f"  Breaches: {breaches}/{len(metrics)}")
        
        lines.append("\n" + "="*60)
        return "\n".join(lines)


# Example: Monitoring
monitor = ValidationMonitor()

# Simulate historical metrics
for i in range(10):
    # Simulate declining quality over time
    pass_rate = 0.98 - (i * 0.01) + np.random.uniform(-0.02, 0.02)
    null_rate = 0.02 + (i * 0.005) + np.random.uniform(-0.01, 0.01)
    
    monitor.record_metric("schema_pass_rate", max(0, min(1, pass_rate)))
    monitor.record_metric("null_rate", max(0, null_rate))

print(monitor.summary())

# Check for breaches
breaches = monitor.get_breaches()
if breaches:
    print(f"\n⚠ {len(breaches)} threshold breaches detected!")
    for b in breaches[-5:]:
        print(f"  - {b.metric_name}: {b.value:.3f} (threshold: {b.threshold})")

---

## 📚 Key Takeaways

### Validation Strategy

| Layer | Purpose | Tools |
|-------|---------|-------|
| **Schema** | Structure, types, constraints | Pydantic, JSON Schema, Custom |
| **Business Rules** | Domain logic, cross-field | Custom rules, dbt tests |
| **Statistical** | Distribution, outliers, drift | SciPy, Custom metrics |
| **Anomaly** | Pattern detection | Isolation Forest, LOF, Mahalanobis |

### Best Practices

1. **Layer Your Validation**
   - Start with fast schema checks
   - Add business rules for domain logic
   - Use statistical tests for drift detection

2. **Choose Appropriate Actions**
   - **GATE**: For critical quality issues that must block processing
   - **ALERT**: For warnings that need attention but shouldn't halt pipelines
   - **QUARANTINE**: For isolating bad records while processing good ones
   - **ENRICH**: For adding metadata for downstream filtering

3. **Monitor Continuously**
   - Track validation metrics over time
   - Set up alerts for threshold breaches
   - Review trends to catch gradual degradation

4. **Document Expectations**
   - Use Great Expectations for self-documenting validation
   - Generate data docs for stakeholder communication
   - Version control your validation rules

### Production Recommendations

```python
# Recommended libraries for production
# pip install great-expectations  # Comprehensive validation
# pip install pandera             # DataFrame validation
# pip install pydantic            # Schema validation
# pip install evidently           # Data drift detection
```

### Validation Checklist

- [ ] Schema validation for all ingested data
- [ ] Business rules documented and tested
- [ ] Statistical baselines established for drift detection
- [ ] Anomaly detection for high-value fields
- [ ] Quarantine process for bad records
- [ ] Monitoring dashboards with alerts
- [ ] Regular review of validation rules